# This is the code which solves the Static Traffic Assignment Problem

In [2]:
import numpy as np

## 1. We load the graph and the demand
Both graph and demand are in csv file

In [3]:
graph = np.loadtxt('braess_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt('braess_od.csv', delimiter=',', skiprows=1)

In [4]:
print(graph)

[[0.   0.   1.   1.   0.1  0.   0.   0.  ]
 [1.   0.   2.   2.   0.   0.   0.   0.  ]
 [2.   1.   2.   0.25 0.   0.   0.   0.  ]
 [3.   1.   3.   2.   0.   0.   0.   0.  ]
 [4.   2.   3.   1.   0.1  0.   0.   0.  ]]


In [5]:
# in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix
try:
    demand.shape[1]
except:
    demand = np.array([demand])
nb_ods = int(demand.shape[0])

In [6]:
demand[0][2] = 10
print(demand)

[[ 0.  3. 10.]]


Then store the links in a dictionary

In [7]:
# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
print(graph_dict)

{0: {1: 0, 2: 1}, 1: {2: 2, 3: 3}, 2: {3: 4}}


Then, we define the function which gives the travel time as a function of the flow

In [8]:
def travel_time(f):
    return graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)

In [10]:
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)

## 2. We compute the all or nothing flow allocation

In [11]:
import scipy

In [18]:
from scipy.sparse.csgraph import dijkstra

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [12]:
def update_travel_time(tt):
    for i in range(graph.shape[0]):
        G[int(graph[i][1])][int(graph[i][2])] = tt[i]

In [13]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
update_travel_time(travel_time(np.zeros(nb_links)))
print(G)

[[0.   1.   2.   0.  ]
 [0.   0.   0.25 2.  ]
 [0.   0.   0.   1.  ]
 [0.   0.   0.   0.  ]]


Now let's compute the all or nothing allocation

In [25]:
# computing the all or nothing flow
def all_or_nothing():
    # using scipy to compute dijkstra
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    faon = np.zeros(shape = nb_links)
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp_d = d_tmp
        while node_tmp_d != o_tmp:
            node_tmp = return_predecessors[o_tmp][node_tmp_d]
            link_tmp = int(graph_dict[node_tmp][node_tmp_d])
            faon[link_tmp] += flow_tmp
            node_tmp_d = node_tmp
    return faon

We define the line search

In [26]:
def potential(graph, f):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    links = int(np.max(graph[:, 0]) + 1)
    g = np.copy(
        graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [27]:
eps=1e-8
f = all_or_nothing()
update_travel_time(travel_time(f))

for i in range(1000):
    faon = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    if s < eps:
        break
    f = (1. - s) * f + s * faon
    update_travel_time(travel_time(f))

In [28]:
print(G)

[[0.         1.75000071 2.         0.        ]
 [0.         0.         0.25       2.        ]
 [0.         0.         0.         1.75000015]
 [0.         0.         0.         0.        ]]


In [30]:
print(f)

[7.50000706 2.49999294 5.00000853 2.49999853 7.50000147]
[1.75000071 2.         0.25       2.         1.75000015]
